# Velr OpenCypher UNWIND Cookbook (BIND tables + Movies demo)


This notebook is a **cookbook of UNWIND patterns that Velr supports today**, based on the
end-to-end tests in the `velr-e2e` crate.

The goals:

- Give you a **copy-pasteable set of UNWIND + BIND patterns**
- Show what **currently works in Velr** in terms of:
  - `UNWIND BIND('<name>') AS alias`
  - Creating nodes and relationships from bind tables
  - Boolean filters on UNWIND columns (`= true`, bare, negation)
  - Multi-source UNWIND joins
  - Lists / JSON collections on nodes and edges
  - MERGE with `ON CREATE` / `ON MATCH` driven by UNWIND
  - Mixed-type columns (TEXT vs JSON) in the same key
  - Global `WHERE` that mixes node + UNWIND props


# 0. Setup


In [ ]:
%pip install velr --force-reinstall 
%pip install pandas polars pyarrow --quiet

In [ ]:
from velr.driver import Velr
import polars as pl
import pandas as pd

db = Velr.open(None)
print("Velr DB opened:", db)

def show(q: str) -> pd.DataFrame:
    """Run a Cypher query and return a pandas DataFrame."""
    return db.to_pandas(q)


# 0.1 Load Movies CSVs from disk (optional)


If you already have the Movies demo CSVs on disk, you can load them and bind
them as in the MATCH cookbook.


In [ ]:
people_csv    = pl.read_csv("../data/movies_people.csv")
movies_csv    = pl.read_csv("../data/movies_movies.csv")
directed_csv  = pl.read_csv("../data/movies_directed.csv")
acted_in_csv  = pl.read_csv("../data/movies_acted_in.csv")

db.bind_polars("_movies_people",   people_csv)
db.bind_polars("_movies_movies",   movies_csv)
db.bind_polars("_movies_directed", directed_csv)
db.bind_polars("_movies_acted_in", acted_in_csv)

print("Bound Movies CSV tables into Velr")


# 0.2 Create Person / Movie nodes via UNWIND BIND(...)


In [ ]:
# Create Person nodes
db.run("""
UNWIND BIND('_movies_people') AS r
CREATE (p:Person {
  key:        r.key,
  name:       r.name,
  born:       r.born,
  birthplace: r.birthplace
});
""")

# Label Actors / Directors / Writers
db.run("""
UNWIND BIND('_movies_people') AS r
MATCH (p:Person {key:r.key})
WHERE r.is_actor
SET p:Actor;
""")

db.run("""
UNWIND BIND('_movies_people') AS r
MATCH (p:Person {key:r.key})
WHERE r.is_director
SET p:Director;
""")

db.run("""
UNWIND BIND('_movies_people') AS r
MATCH (p:Person {key:r.key})
WHERE r.is_writer
SET p:Writer;
""")

# Create Movie nodes
db.run("""
UNWIND BIND('_movies_movies') AS r
CREATE (m:Movie {
  key:      r.key,
  title:    r.title,
  released: r.released,
  imdb:     r.imdb_id,
  runtime:  r.runtime,
  genres:  [r.genre1, r.genre2]
});
""")

# Add one example genre label
db.run("""
UNWIND BIND('_movies_movies') AS r
MATCH (m:Movie {key:r.key})
WHERE r.is_scifi
SET m:ScienceFiction;
""")

print("Movies graph loaded via UNWIND")


# 0.3 Create DIRECTED / ACTED_IN relationships via UNWIND


In [ ]:
# DIRECTED
db.run("""
UNWIND BIND('_movies_directed') AS r
MATCH (d:Person {key:r.director_key}), (m:Movie {key:r.movie_key})
CREATE (d)-[:DIRECTED {since:r.since}]->(m);
""")

# ACTED_IN with both scalar and list props
db.run("""
UNWIND BIND('_movies_acted_in') AS r
MATCH (p:Person {key:r.person_key}), (m:Movie {key:r.movie_key})
CREATE (p)-[:ACTED_IN {
  role:    r.role,
  roles:  [r.role],   // single-element list
  minutes: r.minutes
}]->(m);
""")

print("Relationships created")


# 1. UNWIND → CREATE nodes (JSON + NULL columns)


This pattern corresponds to the `unwind_create_nodes_from_bind_table` test.

We bind a tiny in-memory table with mixed types, then turn each row into a node.


In [ ]:
users = pl.DataFrame({
    "id":   ["u-1", "u-2"],
    "name": ["Alice", "Bob"],
    "meta": [ {"vip": True, "lvl": 1}, {"vip": False} ],
    "note": [ None, "ok" ],
})
db.bind_polars("_users", users)

db.run("""
UNWIND BIND('_users') AS r
CREATE (:User {
  id:   r.id,
  name: r.name,
  meta: r.meta,
  note: r.note
});
""")

show("""
MATCH (u:User)
RETURN u.id AS id, u.name AS name, u.meta AS meta, u.note AS note
ORDER BY id;
""")


# 2. UNWIND + MATCH → CREATE edges (join in WHERE)


Two variants: with the join in `WHERE`, and using node pattern maps only.


In [ ]:
# Seed users/products via UNWIND
users_basic = pl.DataFrame({"id": ["u-1", "u-2"]})
products_basic = pl.DataFrame({"id": ["p-1", "p-2"]})
db.bind_polars("_users_basic", users_basic)
db.bind_polars("_products_basic", products_basic)

db.run("""
UNWIND BIND('_users_basic') AS r
CREATE (:User {id:r.id});
""")

db.run("""
UNWIND BIND('_products_basic') AS r
CREATE (:Product {id:r.id});
""")

# Purchases: user, product, ts
purchases = pl.DataFrame({
    "user":    ["u-1", "u-2"],
    "product": ["p-1", "p-2"],
    "ts":      [100, 200],
})
db.bind_polars("_purchases", purchases)

# Variant 1: join in WHERE
db.run("""
UNWIND BIND('_purchases') AS r
MATCH (u:User), (p:Product)
WHERE u.id = r.user AND p.id = r.product
CREATE (u)-[:BOUGHT {ts: r.ts}]->(p);
""")

show("MATCH ()-[b:BOUGHT]->() RETURN count(b) AS count;")


In [ ]:
# Variant 2: join via node pattern maps (no WHERE)
db.run("""
UNWIND BIND('_purchases') AS r
MATCH (u:User {id:r.user}), (p:Product {id:r.product})
CREATE (u)-[:BOUGHT {ts:r.ts}]->(p);
""")

show("""
MATCH (u:User)-[b:BOUGHT]->(p:Product)
RETURN u.id AS user, p.id AS product, b.ts AS ts
ORDER BY ts;
""")


# 3. Boolean columns from UNWIND in WHERE


Shapes that exercise boolean columns coming directly from UNWIND tables:
- `WHERE r.is_actor = true`
- Bare `WHERE r.is_actor`
- `WHERE r.is_actor = false`
- `WHERE NOT r.is_actor`


In [ ]:
# Reuse Movies people CSV with `is_actor` boolean column
db.run("""
UNWIND BIND('_movies_people') AS r
CREATE (:PersonBool {key:r.key});
""")

db.run("""
UNWIND BIND('_movies_people') AS r
MATCH (p:PersonBool {key:r.key})
WHERE r.is_actor = true
SET p:Actor;
""")

show("MATCH (p:PersonBool:Actor) RETURN count(p) AS actors;")


In [ ]:
db.run("""
UNWIND BIND('_movies_people') AS r
CREATE (:PersonBool2 {key:r.key});
""")

db.run("""
UNWIND BIND('_movies_people') AS r
MATCH (p:PersonBool2 {key:r.key})
WHERE r.is_actor
SET p:Actor;
""")

show("MATCH (p:PersonBool2:Actor) RETURN count(p) AS actors;")


In [ ]:
db.run("""
UNWIND BIND('_movies_people') AS r
CREATE (:PersonFlagged {key:r.key});
""")

db.run("""
UNWIND BIND('_movies_people') AS r
MATCH (p:PersonFlagged {key:r.key})
WHERE r.is_actor = false
SET p:NonActor;
""")

show("MATCH (p:PersonFlagged:NonActor) RETURN count(p) AS non_actors;")


In [ ]:
db.run("""
UNWIND BIND('_movies_people') AS r
CREATE (:PersonFlagged2 {key:r.key});
""")

db.run("""
UNWIND BIND('_movies_people') AS r
MATCH (p:PersonFlagged2 {key:r.key})
WHERE NOT r.is_actor
SET p:NonActor;
""")

show("MATCH (p:PersonFlagged2:NonActor) RETURN count(p) AS non_actors;")


# 4. Two UNWIND sources + join on key


Examples with multiple UNWIND sources, joined on a key and filtered by a boolean flag.


In [ ]:
people_names = pl.DataFrame({
    "key":  ["p-1", "p-2", "p-3"],
    "name": ["Alice", "Bob", "Carol"],
})
people_flags = pl.DataFrame({
    "key":      ["p-1", "p-2", "p-3"],
    "is_actor": [True,  False, True],
})
db.bind_polars("_people_names", people_names)
db.bind_polars("_people_flags", people_flags)

# Seed Person nodes from names only
db.run("""
UNWIND BIND('_people_names') AS n
CREATE (:PersonJoin {key:n.key});
""")

# Read-only join + filter
show("""
UNWIND BIND('_people_names') AS n
UNWIND BIND('_people_flags') AS f
MATCH (p:PersonJoin {key:n.key})
WHERE n.key = f.key AND f.is_actor
RETURN count(p) AS actors_from_join;
""")


In [ ]:
# Mutating version: SET labels and copy name
db.run("""
UNWIND BIND('_people_names') AS n
CREATE (:PersonJoin2 {key:n.key});
""")

db.run("""
UNWIND BIND('_people_names') AS n
UNWIND BIND('_people_flags') AS f
MATCH (p:PersonJoin2 {key:n.key})
WHERE n.key = f.key AND f.is_actor
SET p:Actor, p.name = n.name;
""")

show("""
MATCH (p:PersonJoin2:Actor)
RETURN p.key AS key, p.name AS name
ORDER BY key;
""")


# 5. Lists / JSON collections from UNWIND


Patterns that put list values on nodes and relationships using UNWIND data.


In [ ]:
# Seed Person and Movie nodes
people_simple = pl.DataFrame({"key": ["p-1", "p-2"]})
movies_simple = pl.DataFrame({"key": ["m-1", "m-2"]})
db.bind_polars("_people_simple", people_simple)
db.bind_polars("_movies_simple", movies_simple)

db.run("""
UNWIND BIND('_people_simple') AS r
CREATE (:PersonRoles {key:r.key});
""")

db.run("""
UNWIND BIND('_movies_simple') AS r
CREATE (:MovieRoles {key:r.key});
""")

acted_in = pl.DataFrame({
    "person_key": ["p-1", "p-2"],
    "movie_key":  ["m-1", "m-2"],
    "role":       ["Neo", "Trinity"],
    "minutes":    [120, 110],
})
db.bind_polars("_movies_acted_in_simple", acted_in)

# Edge with list property
db.run("""
UNWIND BIND('_movies_acted_in_simple') AS r
MATCH (p:PersonRoles {key:r.person_key}), (m:MovieRoles {key:r.movie_key})
CREATE (p)-[:ACTED_IN {
  role:    r.role,
  roles:  [r.role],
  minutes: r.minutes
}]->(m);
""")

show("""
MATCH (p:PersonRoles {key:'p-1'})-[a:ACTED_IN]->(m:MovieRoles {key:'m-1'})
RETURN a.role AS role, a.roles AS roles, a.minutes AS minutes;
""")


In [ ]:
# CREATE Person with a list property
roles_df = pl.DataFrame({
    "person_key": ["p-1", "p-2"],
    "role":       ["Neo", "Trinity"],
})
db.bind_polars("_person_roles", roles_df)

db.run("""
UNWIND BIND('_person_roles') AS r
CREATE (:PersonWithRoles {
  key:   r.person_key,
  roles: [r.role]
});
""")

show("""
MATCH (p:PersonWithRoles {key:'p-1'})
RETURN p.roles AS roles;
""")


In [ ]:
# MERGE ACTED_IN edge with ON CREATE SET list
db.run("""
UNWIND BIND('_movies_acted_in_simple') AS r
MATCH (p:PersonRoles {key:r.person_key}), (m:MovieRoles {key:r.movie_key})
MERGE (p)-[a:ACTED_IN]->(m)
ON CREATE SET
  a.role    = r.role,
  a.roles   = [r.role],
  a.minutes = r.minutes;
""")

show("""
MATCH (p:PersonRoles {key:'p-1'})-[a:ACTED_IN]->(m:MovieRoles {key:'m-1'})
RETURN a.role AS role, a.roles AS roles, a.minutes AS minutes;
""")


In [ ]:
# MATCH + SET edge list from UNWIND
db.run("""
UNWIND BIND('_movies_acted_in_simple') AS r
MATCH (p:PersonRoles {key:r.person_key}), (m:MovieRoles {key:r.movie_key})
CREATE (p)-[a:ACTED_IN {role:r.role, minutes:r.minutes}]->(m);
""")

db.run("""
UNWIND BIND('_movies_acted_in_simple') AS r
MATCH (p:PersonRoles {key:r.person_key})-[a:ACTED_IN]->(m:MovieRoles {key:r.movie_key})
SET a.roles = [r.role];
""")

show("""
MATCH (p:PersonRoles {key:'p-1'})-[a:ACTED_IN]->(m:MovieRoles {key:'m-1'})
RETURN a.roles AS roles;
""")


In [ ]:
# SET list on node while creating edge
db.run("""
UNWIND BIND('_movies_acted_in_simple') AS r
MATCH (p:PersonRoles {key:r.person_key}), (m:MovieRoles {key:r.movie_key})
CREATE (p)-[:ACTED_IN {role:r.role, minutes:r.minutes}]->(m)
SET p.roles = [r.role];
""")

show("""
MATCH (p:PersonRoles {key:'p-1'})-[:ACTED_IN]->(m:MovieRoles {key:'m-1'})
RETURN p.roles AS roles;
""")


# 6. MERGE + UNWIND with ON CREATE / ON MATCH


In [ ]:
# MERGE Person with roles list on ON CREATE
person_roles_merge = pl.DataFrame({
    "person_key": ["p-1", "p-2"],
    "role":       ["Neo", "Trinity"],
})
db.bind_polars("_person_roles_merge", person_roles_merge)

db.run("""
UNWIND BIND('_person_roles_merge') AS r
MERGE (p:PersonMerge {key:r.person_key})
ON CREATE SET
  p.roles = [r.role];
""")

show("""
MATCH (p:PersonMerge)
RETURN p.key AS key, p.roles AS roles
ORDER BY key;
""")


In [ ]:
# MERGE + ON MATCH update from UNWIND
init = pl.DataFrame({"person_key": ["p-1"], "role": ["Neo"]})
db.bind_polars("_person_roles_init", init)

db.run("""
UNWIND BIND('_person_roles_init') AS r
MERGE (p:PersonMerge2 {key:r.person_key})
ON CREATE SET
  p.roles = [r.role];
""")

update = pl.DataFrame({"person_key": ["p-1"], "role": ["TheOne"]})
db.bind_polars("_person_roles_update", update)

db.run("""
UNWIND BIND('_person_roles_update') AS r
MERGE (p:PersonMerge2 {key:r.person_key})
ON MATCH SET
  p.roles = [r.role];
""")

show("""
MATCH (p:PersonMerge2 {key:'p-1'})
RETURN p.roles AS roles;
""")


In [ ]:
# MERGE KNOWS between two Persons with lists on both nodes + edge
friends = pl.DataFrame({
    "person_key": ["p-1"],
    "friend_key": ["p-2"],
    "role":       ["Buddy"],
})
db.bind_polars("_friend_roles", friends)

db.run("""
UNWIND BIND('_friend_roles') AS r
MERGE (p:PersonKnow {key:r.person_key})-[a:KNOWS]->(q:PersonKnow {key:r.friend_key})
ON CREATE SET
  p.roles = [r.role],
  q.roles = [r.role],
  a.roles = [r.role];
""")

show("""
MATCH (p:PersonKnow {key:'p-1'})-[a:KNOWS]->(q:PersonKnow {key:'p-2'})
RETURN p.roles AS p_roles, q.roles AS q_roles, a.roles AS a_roles;
""")


In [ ]:
# Two-UNWIND MERGE: real data in the second UNWIND
dummy = pl.DataFrame({"x": [1]})
db.bind_polars("_dummy", dummy)

person_roles_two = pl.DataFrame({
    "person_key": ["p-1", "p-2"],
    "role":       ["Neo", "Trinity"],
})
db.bind_polars("_person_roles_two_unwind", person_roles_two)

db.run("""
UNWIND BIND('_dummy')        AS d
UNWIND BIND('_person_roles_two_unwind') AS r
MERGE (p:PersonFromSecond {key:r.person_key})
ON CREATE SET
  p.roles = [r.role];
""")

show("""
MATCH (p:PersonFromSecond)
RETURN p.key AS key, p.roles AS roles
ORDER BY key;
""")


# 7. Mixed-type UNWIND columns (TEXT vs JSON) for the same key


Confirm that `value_text` and `value_json` buckets behave correctly when a single
column contains both plain text and JSON values.


In [ ]:
mixed_roles = pl.DataFrame({
    "person_key": ["p-1",              "p-2"],
    "role":       ["Neo",              '["Trinity", "T"]'],
})
db.bind_polars("_person_mixed_roles", mixed_roles)

db.run("""
UNWIND BIND('_person_mixed_roles') AS r
CREATE (:PersonMixed {
  key:      r.person_key,
  role_any: r.role
});
""")

show("""
MATCH (p:PersonMixed)
RETURN p.key AS key, p.role_any AS role_any
ORDER BY key;
""")


# 8. Global WHERE mixing node + UNWIND props


Regression shape that splits a global filter into:
- a binds-only part (UNWIND values)
- a per-node part (node properties)


In [ ]:
db.run("""
UNWIND BIND('_movies_people') AS r
CREATE (:PersonGlobal {key:r.key});
""")

db.run("""
UNWIND BIND('_movies_people') AS r
MATCH (p:PersonGlobal {key:r.key})
WHERE r.is_actor = true AND p.key = 'p-1'
SET p:Actor;
""")

show("""
MATCH (p:PersonGlobal:Actor)
RETURN p.key AS key;
""")


# 9. Where to go from here


This UNWIND cookbook is grounded directly in the `tests_unwind.rs` shapes, so all
examples reflect patterns that are currently supported by Velr.

Things you might try next:

- Use `UNWIND` to ingest other datasets (CSV → Polars → BIND → graph)
- Combine UNWIND with the MATCH patterns from the MATCH cookbook
- Explore more complex global filters over UNWIND + node properties
- Experiment with larger bind tables and profile performance

Happy unwinding! 🧶🧠
